Importing Libraries and Defining parameters

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transformers
from torch.autograd import Variable

In [ ]:
input_size = 784        # no. of input neurons (img pixels)
hidden_size = 400       # no. of hidden neurons (img pixels)
out_size = 10           # no. of classes (0-9)
epochs = 10             # no. of times we pass the dataset into the network
batch_size = 100        # input size of the data during one iteration
learning_rate = 0.001   # how fast we are learning

In [ ]:
train_dataset = datasets.MNIST(root='./data', 
                               train=True,
                               transform=transformers.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./data', 
                               train=False,
                               transform=transformers.ToTensor())

In [ ]:
#Make Data iterable by Loading it to a loader
# Shuffle the training data to make it independent of the order
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

Defining the Neural Class

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()
        # define the layers
        self.fc1 = nn.Linear(input_size, hidden_size)       # 1st layer
        self.relu = nn.ReLU()                               # 1st layer activation by ReLU
        self.fc2 = nn.Linear(hidden_size, hidden_size)      # 2nd layer
        self.fc3 = nn.Linear(hidden_size, out_size)         # 3rd layer

    def forward(self, x):
        # forward propogation
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

Create network class and network functions

In [ ]:
# Create an object of the c;ass, which represents our Network
net = Net(input_size, hidden_size, out_size)

# Check if computer has CUDA
CUDA = torch.cuda.is_available()
if CUDA:
    # If it has it, move the network to cuda
    net = net.cuda()
# Define Loss function
# Cross Entropy Loss also comes along with Softmax. Therefore, no nee to specify Softmax separatly
criterion = nn.CrossEntropyLoss()

# Define Optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

Training the Network

In [ ]:
# Visualise the train loader
for i, (images, labels) in enumerate(train_loader):
    print(images.size())
    images = images.view(-1, 784)       # reshape the tensor
    print(images.size())

In [ ]:
# Train the network
correct_train = 0
total_train = 0
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):     # iteration loop that takes a batch of 100 eveytime
        # Flatter img from size (100,1,28,28) to (100,784) and wrap it in a variable
        # (batch size, grayscale, rows, cols)
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()

        # clear gradient
        optimizer.zero_grad()
        outputs = net(images)       # Forward pass the batch of 100 imgs
        _, predicted = torch.max(outputs.data, 1)   # 1st argument: max amount & second argument: index location of each max value
        # _, to igore the first argument ie, outputs.data and return only the second argument as in the index
        total_train += labels.size(0)
        if CUDA:
            correct_train += (predicted.cpu() == labels.cpu()).sum()
        else:
            correct_train += (predicted == labels).sum()

        
        loss = criterion(outputs, labels)       # loss
        loss.backward()                         # back prop
        optimizer.step()                        # update weights

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Iteration [{}/{}], Training Loss: {}, Training Accuracy: {}%'.format(epoch+1, epochs, i+1, len(train_dataset)//batch_size, loss.data, (100*correct_train/total_train)))

print("DONE TRAINING")


Testing the Network

In [ ]:
# In testing, no loss or weight calculation and no updation of weights
correct = 0
total = 0
# no epoch, iterating through all the test images
for images, labels in train_loader:
    images = Variable(images.view(-1, 28*28))
    if CUDA:
        images = images.cuda()
    # For each input, output will have 10 elements
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)         # increment the total count
    
    if CUDA:
        correct += (predicted.cpu() == labels.cpu()).sum()
    else:
        correct += (predicted == labels).sum()

print('Final Test Accuracy: %d %%' % (100 * correct / total))
        